In [21]:
import numpy as np
import torch
from ManhOCR.tool.translate import process_image, translate
import cv2 
from ManhOCR.text_recognition import TextRecognition
from ManhOCR.model.vocab import Vocab
from ManhOCR.tool.config import Cfg

## Infernece normally

In [23]:
text_recog_config_path = './ManhOCR/config/base.yml'
text_recog_weights_path = "./weights/convert_weight_2.pth.tar"
device = torch.device("cpu")
config = Cfg.load_config_from_file(text_recog_config_path)
vocab = Vocab(config["vocab"])

In [3]:
repvgg_text_recog = TextRecognition(device=device, weight_path=text_recog_weights_path, config_path=text_recog_config_path, model_type='repvgg')

In [4]:
img = cv2.imread('./sample/0.png')

In [5]:
repvgg_text_recog.predict(img)

'CỘNG HOÀ XÃ HỘI CHỦ NGHĨA'

In [6]:
cnn = torch.jit.load("./traced_scripts/cnn_model.pt", map_location=device)
encoder = torch.jit.load("./traced_scripts/encoder_model.pt", map_location=device)
decoder = torch.jit.load("./traced_scripts/decoder_model.pt", map_location=device)

In [25]:
def translate(img, max_seq_length=256, sos_token=1, eos_token=2):
    """data: BxCXHxW"""
    device = img.device

    with torch.no_grad():
        src = cnn(img)
        encoder_outputs, hidden = encoder(src)

        translated_sentence = [[sos_token] * len(img)]
        max_length = 0

        while max_length <= max_seq_length and not all(
            np.any(np.asarray(translated_sentence).T == eos_token, axis=1)
        ):
            tgt_inp = torch.LongTensor(translated_sentence).to(device)
            tgt_inp = tgt_inp[-1]
            output, hidden, _ = decoder(tgt_inp, hidden, encoder_outputs)
            output = output.unsqueeze(1)
            output = output.to("cpu")

            values, indices = torch.topk(output, 1)

            indices = indices[:, -1, 0]
            indices = indices.tolist()

            translated_sentence.append(indices)
            max_length += 1

            del output

        translated_sentence = np.asarray(translated_sentence).T

    return translated_sentence

In [26]:
img = cv2.imread('./sample/0.png')
img = process_image(img)
img = np.expand_dims(img, axis=0)
img = torch.FloatTensor(img).to(device=device)

In [28]:
s = translate(img)[0].tolist()
s = vocab.decode(s)
s

'CỘNG HOÀ XÃ HỘI CHỦ NGHĨA'